In [7]:
### This project essentially captures a search feature for audio files.
### The following program allows you to search for a particular sound (series of sound)
### from the file you wanted to search for.

In [8]:
from scipy import fft, arange, signal
from scipy.special import logit, expit
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
from ttictoc import TicToc

In [9]:
def read_file(path):
    """
    Reads the audio .wav file and returns the sample rate and data contents of the file.
    """
    sr, signal = wavfile.read(path)
    return sr, signal[:,0]

In [10]:
def read_all(files):
    """Branch audio file extrapolation. Uses read_file() method."""
    ret = []
    t = TicToc()
    t.tic();
    for fl in files:
        ret.append(read_file(fl))
    t.toc();
    print("Time it took to read data from files of length ", len(files), " = ", round(t.elapsed, 3), " seconds.")
    return np.array(ret)

In [11]:
def frequency_sepectrum(sf, x):
    """
    Derive frequency spectrum of a signal from time domain
    :param x: signal in the time domain
    :param sf: sampling frequency
    :returns frequencies and their content distribution
    """
    x = x - np.average(x)  # zero-centering

    n = len(x)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range

    frqarr = frqarr[range(n // 2)]  # one side frequency range

    x = fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]

    return frqarr, abs(x)

In [12]:
def plot(sample_rate, signal, state=False):
    """
    Method responsible for converting from time domain to frequency
    via the discrete fourier analysis function created in frequency_sepectrum().
    Option to form a plot of the resultant frequency domain graph with its frequency content distribution.
    Returns:
    1. Y: The frequencies.
    2. frq: The content distribution.
    """
    frq, Y = frequency_sepectrum(sample_rate, signal)
    frq = frq
    Y = Y
    if(state == True):
        plt.plot(frq, Y)
    return Y, frq

In [13]:
def fourier_transformation(sample_rates, signals):
    """
    Return Foureir Transformation for each individual plot.
    Note: the first index is reserved for Test file.
    Parameters: 
    1. sample_rates: np.array() array 
    """
    Y = []
    frequencies = []
    for sr,sig in zip(sample_rates, signals):
        content_distribution, frq = plot(sr, sig)
        Y.append(frq)
        frequencies.append(content_distribution)
    return np.array(Y), np.array(frequencies)

In [73]:
def sort_list(lists):
    """
    Sorts a dictionary based on values().
    Return the sorted dictionary. Accepts an unsorted dictionary.
    """
    listofTuples = sorted(lists.items() , reverse=True, key=lambda x: x[1])
    # Iterate over the sorted sequence
    data = []
    for elem in listofTuples :
        data.append((elem[0], elem[1]))  
    return data

In [15]:
def find_deviation(point, tests, titles):
    """
    Finds the deviation from the np.array() 'point' based on individual np.array() 'tests'.
    """
    delta = {}
    t = TicToc()
    t.tic()
    for test, title in zip(tests, titles):
        one_res = percent_calculator(point, test)
        delta[title] = one_res
    t.toc()
    print("Time taken(s): ", round(t.elapsed, 3))
    return sort_list(delta), delta

In [130]:
def percent_calculator(A, B):
    """
    Finds the distribution from SAMPLE based on TEST data.
    Uses jacard similarity.
    """
    intersection = 0
    union = 0
    for a,b in zip(A,B):
        if(round(a) == round(b)):
            intersection += 1
    union = abs(len(A) + len(B) - intersection)
    percent = intersection/float(union)
    return percent

### main() calibration

In [17]:
def retrieve_all_files(path=None):
    """
    Returns a list of audio files with the extension, .wav, as an np.array() object.
    Parameters:
    1. path (current directory if left empty): The path of directory to scrape through. str object.
    """
    import os
    files = os.listdir(path)
    refurbished_list = []
    for file in files:
        if(file.endswith(".wav")):
            refurbished_list.append(file)
    return np.array(refurbished_list)

In [18]:
files = retrieve_all_files()
data = read_all(files)

Time it took to read data from files of length  9  =  0.385  seconds.


In [33]:
def retrieve_sample_signals(data):
    """
    Splits data(above) into signal and sample_rate arrays.
    """
    dsp = []
    sr = []
    for df in data:
        sr.append(df[0])
        dsp.append(df[1])
    return sr, dsp

In [34]:
SR, DSP = retrieve_sample_signals(data)

In [25]:
frequencies, content_distribution = fourier_transformation(SR, DSP)

In [134]:
i = 2
print("Comparing ---", files[i], "--- To everything else.")
result, unsorted_result = find_deviation(content_distribution[i], content_distribution, files)

Comparing --- Animals, batch test 1.wav --- To everything else.
Time taken(s):  8.022


In [135]:
for f in files:
    print(f)

Animals Live Test.wav
Animals,  Shazam Project Test.wav
Animals, batch test 1.wav
Dynoro & Gigi DAgostino - In My Mind.wav
Ed Sheeran - Thinking Out Loud (x Acoustic Session).wav
Ed Sheeran - Thinking Out Loud - (Live At Capitals Jingle Bell Ball 2017).wav
Martin Garrix - Animals (test#1).wav
Marvin Gaye - Lets get it on.wav
Thinking out Loud.wav


In [136]:
result

[('Animals, batch test 1.wav', 1.0),
 ('Animals Live Test.wav', 0.03610654040959191),
 ('Animals,  Shazam Project Test.wav', 0.019080601212973544),
 ('Ed Sheeran - Thinking Out Loud (x Acoustic Session).wav',
  0.013207947792407797),
 ('Dynoro & Gigi DAgostino - In My Mind.wav', 0.01269164798761251),
 ('Martin Garrix - Animals (test#1).wav', 0.011179707033263039),
 ('Ed Sheeran - Thinking Out Loud - (Live At Capitals Jingle Bell Ball 2017).wav',
  0.010606541316016072),
 ('Thinking out Loud.wav', 0.007224504873985058),
 ('Marvin Gaye - Lets get it on.wav', 0.005057723597453788)]

In [137]:
#Reflection... Kinda works, need to work on implementing a better percent modulization curve model.